# fold-rank

## GPU 확인

In [1]:
# ✅ GPU 사용 가능 여부 확인
import torch
import lightgbm as lgb
import xgboost as xgb
import catboost

# 모델 임포트 변경
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

print("✅ PyTorch GPU 사용 가능:", torch.cuda.is_available())
print("✅ LightGBM GPU 지원 여부:", lgb.__file__)
print("✅ XGBoost GPU 지원 여부:", xgb.__version__)
print("✅ CatBoost 버전:", catboost.__version__)

✅ PyTorch GPU 사용 가능: True
✅ LightGBM GPU 지원 여부: /home/ubuntu/.local/lib/python3.12/site-packages/lightgbm/__init__.py
✅ XGBoost GPU 지원 여부: 3.0.0
✅ CatBoost 버전: 1.2.7


## 라이브러리 import

In [2]:
# ✅ 라이브러리 import
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import KFold  
from sklearn.metrics import mean_squared_error 

import warnings
warnings.filterwarnings('ignore')

In [3]:
# seed 고정
sd=42
random.seed(sd)
np.random.seed(sd)
os.environ['PYTHONHASHSEED'] = str(sd)

## 데이터 preprocessing

In [4]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
submission_df = pd.read_csv("data/sample_submission.csv")

In [5]:
train_df = train_df[(train_df['임신 성공 확률'] <= 0.05) | (train_df['임신 성공 확률'] >= 0.95)]

In [6]:
# ID 분리
test_ids = test_df['ID']  # ID 컬럼 따로 저장
train_df.drop(columns=['ID'], inplace=True)
test_df.drop(columns=['ID'], inplace=True)

In [7]:
# 난자 출처 알 수 없음 -> 본인제공 replace
train_df['난자 출처'] = train_df['난자 출처'].replace('알 수 없음', '본인 제공')
test_df['난자 출처'] = test_df['난자 출처'].replace('알 수 없음', '본인 제공')

In [8]:
# Unknown
train_df['세부 시술 유형'] = train_df['세부 시술 유형'].fillna('Unknown')
test_df['세부 시술 유형'] = test_df['세부 시술 유형'].fillna('Unknown')

In [9]:
# train 데이터
train_df.loc[(train_df['미세주입(ICSI) 배아 이식 수'].isna()) & (train_df['시술 유형'] == 'DI'), '미세주입(ICSI) 배아 이식 수'] = 0
train_df.loc[(train_df['미세주입(ICSI) 배아 이식 수'].isna()) & (train_df['시술 유형'] != 'DI'), '미세주입(ICSI) 배아 이식 수'] = -1

# test 데이터
test_df.loc[(test_df['미세주입(ICSI) 배아 이식 수'].isna()) & (test_df['시술 유형'] == 'DI'), '미세주입(ICSI) 배아 이식 수'] = 0
test_df.loc[(test_df['미세주입(ICSI) 배아 이식 수'].isna()) & (test_df['시술 유형'] != 'DI'), '미세주입(ICSI) 배아 이식 수'] = -1

In [10]:
# IVF가 아닌 DI의 경우 결측치 대체
# 설문 조사에서 DI의 경우 대답할 필요가 없는 항목들이 결측치로 들어간 듯
columns_to_update = ['총 생성 배아 수', '이식된 배아 수', '저장된 배아 수',
                    '채취된 신선 난자 수', '수정 시도된 난자 수']

# train_df 처리
for column in columns_to_update:
    if train_df[column].dtype == 'object':
        # object 타입이면 'Not Answer(DI)'로 채우기
        train_df[column] = train_df[column].fillna('Not Answer(DI)')
    elif train_df[column].dtype in ['float64', 'int64']:
        # 숫자 타입이면 0로 채우기
        train_df[column] = train_df[column].fillna(0)

# test_df 처리
for column in columns_to_update:
    if test_df[column].dtype == 'object':
        # object 타입이면 'Not Answer(DI)'로 채우기
        test_df[column] = test_df[column].fillna('Not Answer(DI)')
    elif test_df[column].dtype in ['float64', 'int64']:
        # 숫자 타입이면 0로 채우기
        test_df[column] = test_df[column].fillna(0)

In [11]:
# 경과일 대체
d_col = ['배아 이식 후 경과일']
train_df[d_col] = train_df[d_col].fillna(999)
test_df[d_col] = test_df[d_col].fillna(999)

In [12]:
# 횟수 결측치
cnt_col  = ['이전 총 임신 횟수',  '이전 총 임신 성공 횟수']

train_df[cnt_col] = train_df[cnt_col].fillna('0회')
test_df[cnt_col] = test_df[cnt_col].fillna('0회')

In [13]:
# 횟수, 회 타입 변경
int_col = [ '이전 IVF 시술 횟수', '이전 DI 시술 횟수',
           '이전 총 임신 횟수',  '이전 총 임신 성공 횟수']
    
for col in int_col:
    train_df[col] = train_df[col].astype(str).str.extract('(\d+)').astype(int)
    test_df[col] = test_df[col].astype(str).str.extract('(\d+)').astype(int)

In [14]:
# 조건 : 난자 출처 == "본인 제공" → 난자 기증자 나이를 시술 당시 나이로 설정
condition2 = (train_df['난자 출처'] == "본인 제공")
train_df.loc[condition2, '난자 기증자 나이'] = train_df.loc[condition2, '환자 시술 당시 나이']

condition2_t = (test_df['난자 출처'] == "본인 제공")
test_df.loc[condition2_t, '난자 기증자 나이'] = test_df.loc[condition2_t, '환자 시술 당시 나이']

## 키워드 추출

In [15]:
# 키워드 리스트 정의
unique_types = ['ICSI', 'IVF', 'Unknown', 'IUI', 'FER',
                'BLASTOCYST', 'AH']

# 각 키워드에 대해 새로운 컬럼 생성
for keyword in unique_types:
    train_df[keyword] = train_df['세부 시술 유형'].str.count(keyword)
    test_df[keyword] = test_df['세부 시술 유형'].str.count(keyword)

# 기존 col drop
train_df.drop('세부 시술 유형', axis=1, inplace=True)
test_df.drop('세부 시술 유형', axis=1, inplace=True)

## 파생변수

In [16]:
# 임신을 했는데 출산을 못한경우 = 유산
train_df['유산 횟수'] = train_df['이전 총 임신 횟수'] - train_df['이전 총 임신 성공 횟수']
test_df['유산 횟수'] = test_df['이전 총 임신 횟수'] - test_df['이전 총 임신 성공 횟수']

In [17]:
# 미세주입(ICSI) 아닌 IVF
train_df['미세주입이 아닌 배아 이식 수'] = train_df['이식된 배아 수'] - train_df['미세주입(ICSI) 배아 이식 수']
test_df['미세주입이 아닌 배아 이식 수'] = test_df['이식된 배아 수'] - test_df['미세주입(ICSI) 배아 이식 수']

## 배아 변수

In [18]:
# 단일 배아를 이식한 경우의 이식된 배아 수가 1이면 성공률이 더 높음
train_df.loc[(train_df['단일 배아 이식 여부'] == 1) & (train_df['이식된 배아 수'] == 1), '이식된 배아 수'] = 1.5
test_df.loc[(test_df['단일 배아 이식 여부'] == 1) & (test_df['이식된 배아 수'] == 1), '이식된 배아 수'] = 1.5

In [19]:
# 이식 배아 수/ 나이
# 나이 재 정의
age_bin_mapping = {
    "만18-34세": 1,
    "만35-37세": 2,
    "만38-39세": 3,
    "만40-42세": 4,
    "만43-44세": 5,
    "만45-50세": 6,
    "알 수 없음": 7
}

# 나이  적용
train_df['나이'] = train_df['환자 시술 당시 나이'].map(age_bin_mapping)
test_df['나이'] = test_df['환자 시술 당시 나이'].map(age_bin_mapping)

# 나이 별 배아수
train_df['이식 배아 수/나이'] = train_df['이식된 배아 수'] / train_df['나이']

test_df['이식 배아 수/나이'] = test_df['이식된 배아 수'] / test_df['나이']

# 나이 drop
train_df.drop('나이', axis=1, inplace=True)
test_df.drop('나이', axis=1, inplace=True)

## 범주형

In [20]:
# categorical로 바꿔줌
cat_col = ['배란 자극 시술 여부', '단일 배아 이식 여부', '불임 원인 - 난관 질환', '불임 원인 - 배란 장애', 
           '불임 원인 - 남성 요인', '불임 원인 - 자궁내막증', '불임 원인 - 불명확', '해동 난자 사용 여부', 
           '신선 난자 사용 여부', '동결 배아 사용 여부', '신선 배아 사용 여부','기증 배아 사용 여부', 
           '착상 전 PGD 시행 여부', '착상 전 PGS 시행 여부', 'ICSI', 'IVF', 'Unknown', 'IUI', 'BLASTOCYST',     
           'AH','FER', '유산 횟수']

for col in cat_col:
    train_df[col] = train_df[col].astype(str)
    test_df[col] = test_df[col].astype(str)

In [21]:
# 범주형 변수 지정
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# 범주형 변수를 category 타입으로 변환
for col in categorical_features:
    train_df[col] = train_df[col].astype("category")
    test_df[col] = test_df[col].astype("category")  # 테스트 데이터도 변환

## 모델링

In [22]:
# 타겟 분리
y_train = train_df['임신 성공 확률']
X_train = train_df.drop(columns=['임신 성공 확률'])
X_test = test_df.copy()

# KFold 정의
kf = KFold(n_splits=5, shuffle=True, random_state=sd)

# params 
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'random_state': sd,
    'device': 'gpu'
}

# CatBoost
cat_params = {
    'loss_function': 'RMSE',
    'learning_rate': 0.1,
    'iterations': 1000,
    'depth': 6,
    'random_seed': sd,
    'od_wait': 100,
    'task_type': 'GPU',
    'verbose': 200,
}

# XGBoost
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'learning_rate': 0.1,
    'max_depth': 6,
    'n_estimators': 1000,
    'random_state': sd,
    'tree_method': 'gpu_hist'
}

# 모델별 데이터 준비
models_dict = {'lgbm': [], 'xgb': [], 'cat': []}
preds_dict = {'lgbm': [], 'xgb': [], 'cat': []}
scores_dict = {'lgbm': [], 'xgb': [], 'cat': []}

In [23]:
# 각 모델에서 드롭할 컬럼 지정
# LGBM에서는 drop하지 않는다
drop_features = {
    'lgbm': ['이전 총 임신 성공 횟수', 'BLASTOCYST', 
             '신선 배아 사용 여부','이전 IVF 시술 횟수'],
    'xgb': ['난자 출처'],
    'cat': []
}

# 모델별 데이터 준비
def prepare_model_data(model_name, X_train, X_test):
    X_train_model = X_train.drop(columns=drop_features[model_name])
    X_test_model = X_test.drop(columns=drop_features[model_name])
    categorical_features_model = [col for col in categorical_features if col not in drop_features[model_name]]
    return X_train_model, X_test_model, categorical_features_model

# LGBM 데이터 준비
X_train_lgbm, X_test_lgbm, categorical_features_lgbm = prepare_model_data('lgbm', X_train, X_test)
# XGBoost 데이터 준비
X_train_xgb, X_test_xgb, categorical_features_xgb = prepare_model_data('xgb', X_train, X_test)
# CatBoost 데이터 준비
X_train_cat, X_test_cat, categorical_features_cat = prepare_model_data('cat', X_train, X_test)

In [24]:
from lightgbm import early_stopping, log_evaluation

# LGBM 학습
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train_lgbm)):
    print(f"[LGBM] Fold {fold}")
    X_tr, X_val = X_train_lgbm.iloc[tr_idx], X_train_lgbm.iloc[val_idx]
    y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

    model = LGBMRegressor(**lgb_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(100), log_evaluation(100)],
        categorical_feature=categorical_features_lgbm
    )

    models_dict['lgbm'].append(model)
    val_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    scores_dict['lgbm'].append((rmse, fold))
    preds_dict['lgbm'].append(model.predict(X_test_lgbm))

[LGBM] Fold 0
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 189
[LightGBM] [Info] Number of data points in the train set: 90428, number of used features: 36
[LightGBM] [Info] Using GPU Device: NVIDIA A10, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 12 dense feature groups (0.69 MB) transferred to GPU in 0.003134 secs. 1 sparse feature groups
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 0.245295
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.401239
Did not meet early stopping. Best iteration is:
[91]	valid_0's rmse: 0.401171
[LGBM] Fold 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underli

In [25]:
import xgboost.callback as xcb

# XGBoost 학습
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train_xgb)):
    print(f"[XGBoost-DMatrix] Fold {fold}")
    X_tr, X_val = X_train_xgb.iloc[tr_idx], X_train_xgb.iloc[val_idx]
    y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

    # DMatrix 변환
    dtrain = xgb.DMatrix(X_tr, label=y_tr, enable_categorical=True)
    dvalid = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)
    dtest = xgb.DMatrix(X_test_xgb, enable_categorical=True)

    model = xgb.train(
        params=xgb_params,
        dtrain=dtrain,
        num_boost_round=1000,
        evals=[(dvalid, 'valid')],
        early_stopping_rounds=100,
        verbose_eval=100
    )

    models_dict['xgb'].append(model)
    val_pred = model.predict(dvalid)
    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    scores_dict['xgb'].append((rmse, fold))
    preds_dict['xgb'].append(model.predict(dtest))

[XGBoost-DMatrix] Fold 0
[0]	valid-rmse:0.42671
[100]	valid-rmse:0.40117
[149]	valid-rmse:0.40150
[XGBoost-DMatrix] Fold 1
[0]	valid-rmse:0.42491
[100]	valid-rmse:0.39995
[145]	valid-rmse:0.40048
[XGBoost-DMatrix] Fold 2
[0]	valid-rmse:0.42556
[100]	valid-rmse:0.39970
[146]	valid-rmse:0.40012
[XGBoost-DMatrix] Fold 3
[0]	valid-rmse:0.42641
[100]	valid-rmse:0.40147
[147]	valid-rmse:0.40202
[XGBoost-DMatrix] Fold 4
[0]	valid-rmse:0.42299
[100]	valid-rmse:0.39854
[143]	valid-rmse:0.39890


In [26]:
# CatBoost 학습
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train_cat)):
    print(f"[CatBoost] Fold {fold}")
    X_tr, X_val = X_train_cat.iloc[tr_idx], X_train_cat.iloc[val_idx]
    y_tr, y_val = y_train.iloc[tr_idx], y_train.iloc[val_idx]

    train_pool = Pool(X_tr, label=y_tr, cat_features=categorical_features_cat)
    val_pool = Pool(X_val, label=y_val, cat_features=categorical_features_cat)

    model = CatBoostRegressor(**cat_params)
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=100, verbose=100)

    models_dict['cat'].append(model)
    val_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    scores_dict['cat'].append((rmse, fold))
    preds_dict['cat'].append(model.predict(X_test_cat))

[CatBoost] Fold 0
0:	learn: 0.4254104	test: 0.4271334	best: 0.4271334 (0)	total: 77ms	remaining: 1m 16s
100:	learn: 0.3971569	test: 0.4006611	best: 0.4006603 (99)	total: 2.39s	remaining: 21.3s
200:	learn: 0.3952489	test: 0.4004456	best: 0.4004154 (155)	total: 4.55s	remaining: 18.1s
300:	learn: 0.3937154	test: 0.4003352	best: 0.4003056 (248)	total: 6.05s	remaining: 14s
400:	learn: 0.3923397	test: 0.4003727	best: 0.4002947 (315)	total: 7.5s	remaining: 11.2s
bestTest = 0.4002947289
bestIteration = 315
Shrink model to first 316 iterations.
[CatBoost] Fold 1
0:	learn: 0.4258433	test: 0.4253685	best: 0.4253685 (0)	total: 14.2ms	remaining: 14.2s
100:	learn: 0.3977101	test: 0.3993544	best: 0.3993544 (100)	total: 1.56s	remaining: 13.9s
200:	learn: 0.3955156	test: 0.3989626	best: 0.3989600 (198)	total: 3.04s	remaining: 12.1s
300:	learn: 0.3938207	test: 0.3988972	best: 0.3988964 (282)	total: 4.53s	remaining: 10.5s
400:	learn: 0.3924485	test: 0.3989334	best: 0.3988546 (319)	total: 6.03s	remaining:

## 최종 학습 제출

In [27]:
from scipy.stats import rankdata
import numpy as np

def weighted_rank_ensemble(predictions, weights):
    """가중 랭크 정규화 평균 앙상블"""
    predictions = np.array(predictions)
    ranked_preds = np.array([rankdata(pred) for pred in predictions])
    weighted_avg_rank = np.average(ranked_preds, axis=0, weights=weights)
    return weighted_avg_rank / np.max(weighted_avg_rank)  # 정규화

# 각 모델에 대한 가중치 수동 설정
manual_weights = {
    'lgbm': 1.5,
    'xgb': 3.0,
    'cat': 2.9
}

# 모델당 상위 N개 unique fold만 사용
n = 3  # top-N fold 선택
selected_preds = []
selected_weights = []

for model_name in ['lgbm', 'xgb', 'cat']:
    # RMSE 기준 오름차순 정렬
    folds = sorted(scores_dict[model_name], key=lambda x: x[0])
    unique_fold_idxs = []
    for rmse, fold in folds:
        if fold not in unique_fold_idxs:
            unique_fold_idxs.append(fold)
        if len(unique_fold_idxs) == n:
            break
    print(f"{model_name} 선택된 fold: {unique_fold_idxs}")
    
    for fold_idx in unique_fold_idxs:
        selected_preds.append(preds_dict[model_name][fold_idx])
        selected_weights.append(manual_weights[model_name])

# 최종 앙상블 결과 생성
final_preds = weighted_rank_ensemble(selected_preds, selected_weights)

# 0.3~0.7 사이에 몰려있는 확률 → 약간 더 극단적으로 밀기
clip_final_preds = np.clip(0.89 * final_preds + 0.003, 0, 1)

submission_df['ID'] = test_ids 
submission_df['임신 성공 확률'] = clip_final_preds

file_name = f'new_top{n}_per_model.csv'
submission_df.to_csv(file_name, index=False)
print('파일 저장완료')

lgbm 선택된 fold: [4, 2, 1]
xgb 선택된 fold: [4, 2, 1]
cat 선택된 fold: [4, 2, 1]
파일 저장완료
